In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [ ]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [ ]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Columns: 2877 entries, LINE to X_2875
dtypes: float64(2875), int64(2)
memory usage: 13.1 MB


In [ ]:
from sklearn.model_selection import train_test_split

train_input, vali_input, train_target, vali_target = train_test_split(train_x, train_y, test_size=0.3, random_state=316)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='sigmoid', input_dim=2877))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_95 (Dense)            (None, 64)                184192    
                                                                 
 dense_96 (Dense)            (None, 128)               8320      
                                                                 
 dense_97 (Dense)            (None, 256)               33024     
                                                                 
 dense_98 (Dense)            (None, 512)               131584    
                                                                 
 dense_99 (Dense)            (None, 256)               131328    
                                                                 
 dense_100 (Dense)           (None, 128)               32896     
                                                                 
 dense_101 (Dense)           (None, 32)              

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_input, train_target, epochs=200)
print('Done.')

Epoch 1/200


NotImplementedError: ignored

In [ ]:
preds = model.predict(vali_input)
print('Done.')

6/6 [==============================] - 0s 3ms/step
Done.


In [ ]:
new_preds = list(map(np.argmax, preds))

In [ ]:
np.mean(new_preds == vali_target)

0.65

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['Y_Class'] = preds

In [ ]:
submit.to_csv('./lgbm_submission.csv', index=False)

In [ ]:
from sklearn.neural_network import MLPClassifier

MLP = MLPClassifier().fit(train_input, train_target)

In [ ]:
pred = MLP.predict(vali_input)

In [ ]:
from sklearn.metrics import f1_score

f1_score(average='macro', y_true=vali_target, y_pred=pred)

0.36168565879234954

In [ ]:
f1_score(average='macro', y_true=vali_target, y_pred=new_preds)

0.5290190806099221